In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "prototyping"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2
using PyCall
using Distances

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMolNotebooks
  ✓ MeshIO
  ✓ CairoMakie
  ✓ MorphoMol
  ✓ MorphoMolNotebooks
  4 dependencies successfully precompiled in 117 seconds. 280 already precompiled.
[ Info: Precompiling MorphoMolNotebooks [3c5b58fc-d541-4bcd-a332-95372d8cdad8]


In [2]:
gauss_kernel(x, positions, charges, b) = sum([charges[i] * exp(-b*euclidean(x, positions[i])^2) for i in 1:length(positions)])

gauss_kernel (generic function with 1 method)

In [3]:
charges = Vector{Float64}([])
positions = Vector{Vector{Float64}}([])
for line in readlines("assets/pqrs/6r7m_parse.pqr")
    line_array = split(line)
    if line_array[1] ==  "ATOM"
        p = [parse(Float64, line_array[6]), parse(Float64, line_array[7]), parse(Float64, line_array[8])]
        q = parse(Float64, line_array[9])
        push!(positions, p)
        push!(charges, q)
    end
end

radii = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY["6r7m"]["template_radii"]
centers = MorphoMol.Utilities.get_point_vector_realization([0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ,MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY["6r7m"]["template_centers"])
applied_charges = [gauss_kernel(c, positions, charges, 10.0) for c in centers]

1206-element Vector{Float64}:
 -0.7799671898303852
 -4.378514768498838e-10
  0.5499998345028063
 -0.5499998434886023
 -9.617756625450913e-10
 -0.4899776264228722
 -0.399992311711711
 -2.0757119854693957e-10
  0.5499998531886483
 -0.5499998620724103
  0.12499999998392412
 -0.12500000107557713
 -0.12499473112229083
  ⋮
 -3.4837582912981367e-9
 -0.12499415410405833
 -0.12499488522303621
 -0.12499415416683705
 -0.12499473189204259
 -0.39999195222672956
 -2.0262976397317859e-10
  0.5499997411441514
 -0.4899829374927875
 -6.128873963044401e-10
 -0.4899802281211102
 -3.5642816394569596e-26

In [4]:
c_plus = (0.8, 0.2, 0.0)
c_minus = (0.0, 0.2, 0.8)
c_neutral = (0.2, 0.2, 0.2)

cutoff = 0.1
colors = [c_neutral for i in 1:length(applied_charges)]
labels = [0 for i in 1:length(applied_charges)]
for i in 1:length(applied_charges)
    if charges[i] > cutoff
        colors[i] = c_plus
        labels[i] = 1
    elseif charges[i] < -cutoff
        colors[i] = c_minus
        labels[i] = 2
    end
end

MorphoMolNotebooks.configuration_to_poly(centers, radii, colors, "assets/6r7m_parse_charges")

In [5]:
function get_chromatic_persistence(points, labels)
    py"""
    import chromatic_tda as chro
    def get_chromatic_persistence(points, labels):
        chro_alpha = chro.ChromaticAlphaComplex(points, labels) 
        # these options make sense for three colors; for two use, e.g., just sub_complex='mono-chromatic'
        simplicial_complex = chro_alpha.get_simplicial_complex(sub_complex="bi-chromatic", full_complex="all", relative="mono-chromatic")  
        six_pack = simplicial_complex.bars_six_pack()
        #chro.plot_six_pack(six_pack)
        return six_pack
    """
    py"get_chromatic_persistence"(points, labels)
end

get_chromatic_persistence (generic function with 1 method)

In [6]:
get_chromatic_persistence(centers, labels)

Dict{Any, Any} with 6 entries:
  "sub_complex" => Dict{Any, Any}(2=>[(1.15351, 1.32543), (1.17379, 1.1827), (1…
  "complex"     => Dict{Any, Any}(2=>[(1.15351, 1.32543), (1.17379, 1.1827), (1…
  "relative"    => Dict{Any, Any}(4=>[(1.79973, 1.80015), (1.82871, 1.8346), (1…
  "cokernel"    => Dict{Any, Any}(2=>[(1.17454, 1.18131), (1.2392, 1.23968), (1…
  "kernel"      => Dict{Any, Any}(2=>[(1.1832, 1.18652), (1.49408, 1.49425), (1…
  "image"       => Dict{Any, Any}(2=>[(1.15351, 1.32543), (1.17379, 1.1827), (1…